## Meta

Comp:
https://www.kaggle.com/c/digit-recognizer    

Reference kernel:
https://www.kaggle.com/kanncaa1/pytorch-tutorial-for-deep-learning-lovers

## Imports

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline 
%load_ext autotime

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing
import torch

import os

time: 527 ms


## Basics

Convert Numpy array to torch and vice versa

In [9]:
# random numpy array
array = np.random.rand(2,2)
print("{} {}\n".format(type(array),array))

# from numpy to tensor
from_numpy_to_tensor = torch.from_numpy(array)
print("{}\n".format(from_numpy_to_tensor))

# from tensor to numpy
tensor = from_numpy_to_tensor
from_tensor_to_numpy = tensor.numpy()
print("{} {}\n".format(type(from_tensor_to_numpy),from_tensor_to_numpy))

<class 'numpy.ndarray'> [[0.52093985 0.39737162]
 [0.31018405 0.02393645]]

tensor([[0.5209, 0.3974],
        [0.3102, 0.0239]], dtype=torch.float64)

<class 'numpy.ndarray'> [[0.52093985 0.39737162]
 [0.31018405 0.02393645]]

time: 7.9 ms
